# Extraccion de datos

In [2]:
import psycopg2
import pandas as pd
from query_sql import query_POSTGRESQL 
import os, sys
import xml.etree.ElementTree as ET

#%% RUTAS

path_files = os.getcwd()
# path_sql = os.path.join(os.path.normpath(os.getcwd() + os.sep + os.pardir), "funciones_sql") 
path_sql = os.path.join(os.path.normpath(os.getcwd() + os.sep + os.pardir), "Tarea2") 


#%% IMPORT DE FUNCIONES

sys.path.insert(0, path_sql)
from query_sql import query_POSTGRESQL, DataFrame2DataBase

#%% EXTRACCION DE DATOS

query_sql = '''SELECT * FROM dtes'''
df = query_POSTGRESQL(path_sql, 'USM', query_sql)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535 entries, 0 to 534
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            535 non-null    int64 
 1   rutemisor     535 non-null    object
 2   folio         535 non-null    int64 
 3   date          535 non-null    object
 4   xml_dte       535 non-null    object
 5   convert_from  535 non-null    object
dtypes: int64(2), object(4)
memory usage: 25.2+ KB


In [4]:
df.head(1)

,id,rutemisor,folio,date,xml_dte,convert_from
0,99760,775668008,440,2024-01-05,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iSVNPLT...,"<?xml version=""1.0"" encoding=""ISO-8859-1""?>\n<..."


# Transformacion de datos

In [18]:
ns = {'ns': 'http://www.sii.cl/SiiDte'}

# Funcion para obtener un elemento
def get_element_value(element, tag):
    return element.find(f'ns:{tag}', ns).text if element.find(f'ns:{tag}', ns) is not None else None

# Funcion para combinar columnas
def combinar_columnas(df, col1, col2, nueva_columna):
    df[nueva_columna] = df[col1].astype(str) + " " + df[col2].astype(str)
    df = df.drop(columns=[col1, col2])
    return df

# Funcion para cambiar de posicion columnas
def cambiar_lugar_columnas(df, col1, col2):
    if col1 in df.columns and col2 in df.columns:
        cols = list(df.columns)
        col1_index, col2_index = cols.index(col1), cols.index(col2)
        
        # Intercambiar los nombres de las columnas en la lista
        cols[col1_index], cols[col2_index] = cols[col2_index], cols[col1_index]
        
        # Reordenar el DataFrame según la nueva lista de columnas
        df = df[cols]
        print(f"Columnas '{col1}' y '{col2}' han sido intercambiadas.")
    else:
        print(f"Una o ambas columnas '{col1}' y '{col2}' no existen en el DataFrame.")
    return df


filas_sin = []
filas_con = []

# Recorrer el dataframe
for xml in df['convert_from']:

    root = ET.fromstring(xml)
    namespace = {'default': 'http://www.sii.cl/SiiDte'}

    # Obtener los valores "globales" del xml
    Folio = root.find('.//{http://www.sii.cl/SiiDte}Folio').text
    RutEmisor = root.find('.//{http://www.sii.cl/SiiDte}RutEmisor').text

    # Obtener todos los detalles del xml <detalle>
    detalles = root.findall('.//ns:Detalle', ns)

    # Recorrer cada detalle de los detalles
    for detalle in detalles:
        NroLinDet = get_element_value(detalle, 'NroLinDet')
        NmbItem = get_element_value(detalle, 'NmbItem')
        MontoItem = get_element_value(detalle, 'MontoItem')
        DscItem = get_element_value(detalle, 'DscItem')
        # Crear una fila para agregar al dataframe
        # Si esta el elemento <DscItem> se agrega al df que contiene las intancias con DscItem
        if DscItem is not None:
            nuevaFila = {
            "rut_emisor": RutEmisor,
            "folio": Folio,
            "n_item":NroLinDet,
            "nmb_item": NmbItem,
            "dsc_item": DscItem,
            "monto_item": MontoItem,
            }
            filas_con.append(nuevaFila)
        # Si no esta el elemento <DscItem> se agrega la fila al df que no tiene 
        else:
            nuevaFila = {
            "rut_emisor": RutEmisor,
            "folio": Folio,
            "n_item":NroLinDet,
            "nmb_item": NmbItem,
            "dsc_item": None,
            "monto_item": MontoItem,
            }
            filas_sin.append(nuevaFila)

# agregar las nuevas filas a los distintos df
df_con = pd.DataFrame(filas_con) # df con DscItem
df_sin = pd.DataFrame(filas_sin) # df sin DscItem

In [19]:
df_con.head()

,rut_emisor,folio,n_item,nmb_item,dsc_item,monto_item
0,77566800-8,440,1,Arriendo local 2 Puerto,montt valor proporcional mes de Enero uf 140,5156309
1,76124336-5,1926,1,ARRIENDO ENERO 2024,VILLAGRAN 571 MULCHEN,1104923
2,8703487-9,57,1,ARRIENDO PROPORCIONAL,"MES DE ENERO 2024, GALPON LOTE A-3-VII-7",2821238
3,76160214-4,10774,2,IDO:348 c. CONSULTAS DE PREVALIDACIÓN SIN SOLI...,IDO:348 : 2% DE DESCUENTO - Incidencias Diciem...,44688
4,97006000-6,2149328,1,COMISION BOLETAS GARANTIA EXENTAS NUMERO : D09...,A : PESOS VCTO : 22/06/2026,865


In [20]:
# Elimino la columna DscItem porque no tiene nada en ese campo
df_sin = df_sin.drop(columns=['dsc_item'])
# Renombro las columna NmbItem que es la unica glosa que hay
df_sin = df_sin.rename(columns={'nmb_item': 'glosa'})

In [21]:
df_con = combinar_columnas(df_con,'nmb_item','dsc_item','glosa')
df_con = cambiar_lugar_columnas(df_con, 'monto_item', 'glosa')

Columnas 'monto_item' y 'glosa' han sido intercambiadas.


In [22]:
df_new = pd.concat([df_con, df_sin], axis=0, ignore_index=False)


In [23]:
df_new.head(10)

,rut_emisor,folio,n_item,glosa,monto_item
0,77566800-8,440,1,Arriendo local 2 Puerto montt valor proporc...,5156309
1,76124336-5,1926,1,ARRIENDO ENERO 2024 VILLAGRAN 571 MULCHEN,1104923
2,8703487-9,57,1,"ARRIENDO PROPORCIONAL MES DE ENERO 2024, GALPO...",2821238
3,76160214-4,10774,2,IDO:348 c. CONSULTAS DE PREVALIDACIÓN SIN SOLI...,44688
4,97006000-6,2149328,1,COMISION BOLETAS GARANTIA EXENTAS NUMERO : D09...,865
5,97006000-6,2149329,1,COMISION BOLETAS GARANTIA EXENTAS NUMERO : D09...,194
6,97006000-6,2149331,1,COMISION BOLETAS GARANTIA EXENTAS NUMERO : D09...,51375
7,97006000-6,2149330,1,COMISION BOLETAS GARANTIA EXENTAS NUMERO : D09...,85
8,97006000-6,2149334,1,COMISION BOLETAS GARANTIA EXENTAS NUMERO : D09...,5771
9,97006000-6,2149333,1,COMISION BOLETAS GARANTIA EXENTAS NUMERO : D09...,2583


In [24]:
df_new.index.name = 'id'  # Nombrar el índice

In [25]:
df_new = df_new.reset_index()

In [26]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1102 entries, 0 to 1101
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1102 non-null   int64 
 1   rut_emisor  1102 non-null   object
 2   folio       1102 non-null   object
 3   n_item      1102 non-null   object
 4   glosa       1102 non-null   object
 5   monto_item  1102 non-null   object
dtypes: int64(1), object(5)
memory usage: 51.8+ KB


In [27]:
# CARGA DE DATOS

# table_name = 'mendoza_bustos_fonseca_ramos'
# DataFrame2DataBase(path_sql, 'USM', df_new, table_name, 'python')